In [1]:
import pandas as pd 
import numpy as np
import os
import sys
from collections import defaultdict

import warnings

warnings.filterwarnings("ignore")

In [2]:
train_data_path = "/opt/ml/code/input/train.csv"
test_data_path = "/opt/ml/code/input/sample_submission.csv"

raw_train_data = pd.read_csv(train_data_path, parse_dates=['order_date'])
test_data = pd.read_csv(test_data_path)

In [3]:
raw_train_data['year_month'] = raw_train_data['order_date'].dt.strftime('%Y-%m')

In [4]:
customer_id_list = pd.unique(raw_train_data.customer_id)
month_list = pd.unique(raw_train_data.year_month)

In [5]:
r_data = raw_train_data.groupby(["customer_id", "year_month"])["total"].sum()

In [6]:
raw_data_ = pd.DataFrame({'total' : r_data}).reset_index()
raw_data_.head(10)

,customer_id,year_month,total
0,12346,2009-12,187.2750
1,12346,2010-01,-22.2750
2,12346,2010-03,44.6325
3,12346,2010-06,-285.9285
4,12346,2010-10,-9.0750
5,12346,2011-01,0.0000
6,12347,2010-10,1009.0245
7,12347,2010-12,1174.4535
8,12347,2011-01,784.3935
9,12347,2011-04,1049.8125


In [7]:
insert_list = []
not_bought_list = []
for customer_id in customer_id_list:
    for month in month_list:
        if raw_data_[raw_data_.customer_id == customer_id]["year_month"].isin([month]).any():
            if customer_id == 12346:
                print(month)
            pass
        else:
            temp = {}
            temp["customer_id"] = customer_id
            temp["year_month"] = month
            temp["total"] = (0.0)
            insert_list.append(temp)
            not_bought_list.append((customer_id, month))


2009-12
2010-01
2010-03
2010-06
2010-10
2011-01


In [8]:
raw_data_ = raw_data_.append(insert_list, ignore_index=True)

In [9]:
raw_data_.tail(10)

,customer_id,year_month,total
141926,17911,2011-01,0.0
141927,17911,2011-02,0.0
141928,17911,2011-03,0.0
141929,17911,2011-04,0.0
141930,17911,2011-05,0.0
141931,17911,2011-06,0.0
141932,17911,2011-07,0.0
141933,17911,2011-08,0.0
141934,17911,2011-09,0.0
141935,17911,2011-10,0.0


In [10]:
month_sorted_list = ['12']
for month in range(1, 12):
    month_sorted_list.append(str(month).zfill(2))

In [11]:
raw_data_[(raw_data_.customer_id == 12346)]

,customer_id,year_month,total
0,12346,2009-12,187.2750
1,12346,2010-01,-22.2750
2,12346,2010-03,44.6325
3,12346,2010-06,-285.9285
4,12346,2010-10,-9.0750
5,12346,2011-01,0.0000
38664,12346,2010-02,0.0000
38665,12346,2010-04,0.0000
38666,12346,2010-05,0.0000
38667,12346,2010-07,0.0000


In [14]:
from tqdm import tqdm

In [15]:
train_year = 2010

train_raw_data = {"customer_id": [], 
                  "label": [],
                  "last_bought": [], 
                  "first_bought": [],
                  "thres_3": [],
                  "thres_6": [],
                  "thres_12": []}

for month in month_sorted_list:      
    train_raw_data[month] = []

for customer_id in tqdm(customer_id_list):
    bought_count = [0] * 3
    bought_more_than_thres = [0] * 3
    first_bought, last_bought = None, None
    for idx, month in enumerate(month_sorted_list):
        if month == '12':
            label = str(train_year - 1)
        else:
            label = str(train_year)
        label += "-" + month
        
        total = float(raw_data_[(raw_data_.customer_id == customer_id) & (raw_data_.year_month == label)]["total"])
        train_raw_data[month].append(total)
        
        if (customer_id, label) not in not_bought_list:
            if first_bought == None:
                first_bought = idx
            last_bought = idx
            bought_count[0] += 1.
            if idx >= 6:
                bought_count[1] += 1.
            if idx >= 9:
                bought_count[2] += 1.
            if total >= 300.:
                bought_more_than_thres[0] += 1.
                if idx >= 6:
                    bought_more_than_thres[1] += 1.
                if idx >= 9:
                    bought_more_than_thres[2] += 1.
    if first_bought == None:
        first_bought = -1
    if last_bought == None:
        last_bought = -1
    train_raw_data["customer_id"].append(customer_id)
    train_raw_data["thres_3"].append(bought_more_than_thres[2] / bought_count[2] if bought_count[2] > 0 else 0.)
    train_raw_data["thres_6"].append(bought_more_than_thres[1] / bought_count[1] if bought_count[1] > 0 else 0.)
    train_raw_data["thres_12"].append(bought_more_than_thres[0] / bought_count[0] if bought_count[0] > 0 else 0.)
    train_raw_data["last_bought"].append(last_bought)
    train_raw_data["first_bought"].append(first_bought)
    train_raw_data["label"].append(1 if float(raw_data_[(raw_data_.customer_id == customer_id) & (raw_data_.year_month == "2010-12")]["total"]) >= 300. else 0)

100%|██████████| 5914/5914 [20:24<00:00,  4.83it/s]


In [16]:
train_data = pd.DataFrame(train_raw_data)
train_data.to_csv(os.path.join("/opt/ml/code/my_src/data" , 'train_data_thres_rate_3_6_12.csv'), index=False)

In [35]:
test_year = 2011

test_raw_data = {"customer_id": [], 
                  "last_bought": [], 
                  "first_bought": [],
                  "thres_3": [],
                  "thres_6": [],
                  "thres_12": []}

for month in month_sorted_list:      
    test_raw_data[month] = []

for customer_id in tqdm(customer_id_list):
    bought_count = [0] * 3
    bought_more_than_thres = [0] * 3
    first_bought, last_bought = None, None
    for idx, month in enumerate(month_sorted_list):
        if month == '12':
            label = str(test_year - 1)
        else:
            label = str(test_year)
        label += "-" + month
        
        total = float(raw_data_[(raw_data_.customer_id == customer_id) & (raw_data_.year_month == label)]["total"])
        test_raw_data[month].append(total)
        
        if (customer_id, label) not in not_bought_list:
            if first_bought == None:
                first_bought = idx
            last_bought = idx
            bought_count[0] += 1.
            if idx >= 6:
                bought_count[1] += 1.
            if idx >= 9:
                bought_count[2] += 1.
            if total >= 300.:
                bought_more_than_thres[0] += 1.
                if idx >= 6:
                    bought_more_than_thres[1] += 1.
                if idx >= 9:
                    bought_more_than_thres[2] += 1.
    if first_bought == None:
        first_bought = -1
    if last_bought == None:
        last_bought = -1
    test_raw_data["customer_id"].append(customer_id)
    test_raw_data["thres_3"].append(bought_more_than_thres[2] / bought_count[2] if bought_count[2] > 0 else 0.)
    test_raw_data["thres_6"].append(bought_more_than_thres[1] / bought_count[1] if bought_count[1] > 0 else 0.)
    test_raw_data["thres_12"].append(bought_more_than_thres[0] / bought_count[0] if bought_count[0] > 0 else 0.)
    test_raw_data["last_bought"].append(last_bought)
    test_raw_data["first_bought"].append(first_bought)
    

100%|██████████| 5914/5914 [21:05<00:00,  4.67it/s]


In [21]:
del test_raw_data["label"]

In [36]:
test_data = pd.DataFrame(test_raw_data)
test_data.to_csv(os.path.join("/opt/ml/code/my_src/data" , 'test_data_thres_rate_3_6_12.csv'), index=False)

In [23]:
train_csv = pd.read_csv("/opt/ml/code/my_src/data/train_data_thres_rate_3_6_12.csv")

In [25]:
train_csv.head(10)

,customer_id,label,last_bought,first_bought,thres_3,thres_6,thres_12,12,01,02,03,04,05,06,07,08,09,10,11
0,13085,0,10,0,0.000000,0.000000,0.666667,1809.7200,1518.6600,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-1369.6980,0.0000
1,13078,1,11,0,1.000000,1.000000,1.000000,3759.1950,1412.3175,1452.066,2168.4795,1157.1780,1686.5805,2003.0175,1634.3580,2110.3500,1830.5925,2302.7895,4779.3900
2,15362,0,9,0,1.000000,1.000000,1.000000,512.7375,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,498.8445,0.0000,0.0000
3,18102,1,11,0,0.666667,0.833333,0.916667,67659.4710,63616.2450,36756.885,35004.9315,27384.3075,21737.5125,25326.8730,78781.0815,76752.0435,-267.3000,48799.8060,36452.6415
4,12682,1,11,0,1.000000,1.000000,1.000000,1770.3675,1409.6775,2768.073,1137.5925,1425.0060,979.2585,2070.7830,657.5910,756.8055,1429.0650,817.0800,2239.4955
5,18087,0,11,0,1.000000,1.000000,1.000000,1633.0710,502.8540,512.820,401.7750,517.9680,0.0000,0.0000,0.0000,9466.5780,504.9000,0.0000,4018.5090
6,13635,0,9,0,1.000000,1.000000,1.000000,1296.9825,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1800.3810,0.0000,0.0000
7,14110,0,9,0,1.000000,1.000000,1.000000,1116.2250,502.3755,0.000,1030.1280,1594.3290,2309.8350,1517.3565,1048.1790,1007.2590,1899.4305,0.0000,0.0000
8,12636,0,0,0,0.000000,0.000000,0.000000,232.6500,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,17519,1,9,0,1.000000,1.000000,1.000000,508.9260,498.9600,477.840,544.9785,531.8775,677.1600,540.0450,545.3745,0.0000,506.7480,0.0000,0.0000


In [37]:
test_csv = pd.read_csv("/opt/ml/code/my_src/data/test_data_thres_rate_3_6_12.csv")

In [38]:
test_csv.head(10)

,customer_id,last_bought,first_bought,thres_3,thres_6,thres_12,12,01,02,03,04,05,06,07,08,09,10,11
0,13085,7,2,0.0,0.0,0.333333,0.0000,0.0000,458.865,0.0000,-237.1050,0.0000,0.0000,227.667,0.000,0.0000,0.0000,0.000
1,13078,11,0,1.0,1.0,1.000000,1440.8790,1230.8670,553.443,2028.7080,1165.0320,748.8525,1281.0930,925.947,1799.919,2467.5750,2317.8870,3190.836
2,15362,-1,-1,0.0,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0000,0.000
3,18102,11,0,1.0,1.0,1.000000,45927.1065,0.0000,17383.542,11400.3780,1713.9375,29973.4710,69233.0760,32817.114,1626.768,81876.5970,86924.0955,25296.282
4,12682,11,0,1.0,1.0,1.000000,1774.2285,1377.9315,1652.871,791.0100,966.1575,2145.3300,1295.3985,1688.709,1871.232,955.8285,703.8570,3988.710
5,18087,9,2,1.0,0.5,0.666667,0.0000,0.0000,5217.498,0.0000,0.0000,0.0000,0.0000,-403.920,0.000,1407.1200,0.0000,0.000
6,13635,10,10,1.0,1.0,1.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,1767.1995,0.000
7,14110,11,3,1.0,1.0,0.888889,0.0000,0.0000,0.000,1492.0290,-22.2750,1315.4790,626.5380,523.710,1420.617,1494.8835,1361.3490,605.616
8,12636,-1,-1,0.0,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0000,0.000
9,17519,11,0,1.0,1.0,1.000000,545.0775,0.0000,0.000,517.1925,0.0000,506.3025,330.8250,547.635,0.000,0.0000,0.0000,587.433


In [ ]:
#0.8594
#recent/ 소비자별 3 개월, 6 개월 별로 300 달러는 넘는 비율 1 년단위 최근에 산날 min max /product id의 라벨링코드 